In [4]:
import pandas as pd
import datetime

In [5]:
df_vp = pd.read_parquet("/home/donsson/proyectos/API/ventas_perdidas_2025.parquet")

# REGLAS CONTAR VP

In [6]:
import pandas as pd
import numpy as np

# ===============================
# Filtrar almacenamiento agotado
# ===============================
df_vp = df_vp[df_vp["almacenamiento_tipo"].str.lower() == "agotado"]

# ===============================
# Asegurar tipos correctos
# ===============================
df_vp = df_vp.copy()
df_vp["fecha"] = pd.to_datetime(df_vp["fecha"], errors="coerce")

# Numéricos
for col in ["cantidad", "cantidad_existencia", "cantidad_reservada"]:
    df_vp[col] = pd.to_numeric(df_vp[col], errors="coerce").fillna(0).clip(lower=0)

# ===============================
# Reglas Odoo vectorizadas
# ===============================
is_cot = df_vp["origen"].fillna("").str.lower() == "cotizacion"
ignore_mask = df_vp["cantidad"] >= 100

ajuste = np.where(
    is_cot,
    df_vp["cantidad"] - df_vp["cantidad_existencia"] - df_vp["cantidad_reservada"],
    df_vp["cantidad"] - df_vp["cantidad_reservada"]
)

# Aplicar reglas de descarte y piso en cero
ajuste = np.where(ignore_mask, 0, ajuste)
ajuste = np.where(ajuste > 0, ajuste, 0)

df_vp["ventas_perdidas"] = ajuste.astype(float)

# ===============================
# Columnas temporales
# ===============================
df_vp["Semana"] = df_vp["fecha"].dt.to_period("W").dt.start_time
df_vp["ano"]   = df_vp["Semana"].dt.year
df_vp["mes"]   = df_vp["Semana"].dt.month
df_vp["dia"]   = df_vp["Semana"].dt.day

# ===============================
# Filtro adicional: excluir SERV y CARCASA
# ===============================
mask_excluir = ~df_vp["product_ref"].str.contains("SERV|CARCASA", case=False, na=False)
df_vp = df_vp[mask_excluir]

# ===============================
# Agrupación por tienda + producto + semana
# ===============================
lost_by_week = (
    df_vp.groupby(["store_name", "product_ref", "Semana", "ano", "mes", "dia"])
    .agg(
        lost_sales=("ventas_perdidas", "sum"),   # suma total de ventas perdidas
        veces_vp=("ventas_perdidas", "count")    # número de veces que hubo pérdida
    )
    .reset_index()
)

# Mostrar resultado agrupado
vp_week = lost_by_week


vp_week.sample(20)

,store_name,product_ref,Semana,ano,mes,dia,lost_sales,veces_vp
26475,SUCURSAL CALLE 6,BCS00298125,2025-07-28,2025,7,28,1.0,1
2618,PRINCIPAL COTA,BHE00826125,2025-08-25,2025,8,25,1.0,1
20489,SUCURSAL BUCARAMANGA,DAE04023025,2025-02-10,2025,2,10,1.0,1
15123,SUCURSAL BARRANQUILLA,DAP09109025,2025-03-03,2025,3,3,2.0,2
11289,SUCURSAL BARRANQUILLA,BCS20364125,2025-09-15,2025,9,15,1.0,1
19939,SUCURSAL BUCARAMANGA,DAB09178025,2025-01-20,2025,1,20,1.0,1
34723,SUCURSAL MEDELLIN,DAB08069025,2025-05-19,2025,5,19,1.0,1
42212,SUCURSAL VALLADOLID,BLS00029125,2025-02-10,2025,2,10,3.0,1
11978,SUCURSAL BARRANQUILLA,BHS00606125,2025-04-07,2025,4,7,1.0,1
33972,SUCURSAL MEDELLIN,BLS00068125,2025-07-14,2025,7,14,1.0,1


In [8]:
vp_week.to_excel("/home/donsson/proyectos/MODELO ABASTECIMIENTO/exceles/vp_año.xlsx")

In [11]:
mes = "TODOS"
vp = vp_week[(vp_week["ano"]==2025)]
vp.to_excel(f"/home/donsson/proyectos/MODELO ABASTECIMIENTO/exceles/vp_meses/vp_definitivasparaanalisis{mes}.xlsx")


In [10]:
vp.head(20)

,store_name,product_ref,Semana,ano,mes,dia,lost_sales,veces_vp
0,PRINCIPAL COTA,AC10388020,2025-09-22,2025,9,22,30.0,2
56,PRINCIPAL COTA,ALB115W4050,2025-09-01,2025,9,1,2.0,2
57,PRINCIPAL COTA,ALB115W4050,2025-09-15,2025,9,15,1.0,1
58,PRINCIPAL COTA,ALB115W4050,2025-09-22,2025,9,22,1.0,1
80,PRINCIPAL COTA,ALB15W40030,2025-09-15,2025,9,15,3.0,2
81,PRINCIPAL COTA,ALB15W40030,2025-09-22,2025,9,22,1.0,1
88,PRINCIPAL COTA,ALB15W40052,2025-09-01,2025,9,1,1.0,1
94,PRINCIPAL COTA,ALG10W30030,2025-09-08,2025,9,8,1.0,1
121,PRINCIPAL COTA,ALG15W40131,2025-09-01,2025,9,1,0.0,1
141,PRINCIPAL COTA,ALQ110W3030,2025-09-08,2025,9,8,1.0,1
